In [1]:
import gensim.downloader as api
from sklearn.datasets import fetch_20newsgroups
import torch

from experiments.experiment import snn_experiment, lsa_experiment
from experiments.preprocess import Word2VecPrep

In [2]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [3]:
word2vec = api.load('word2vec-google-news-300')

In [4]:
cats = ['comp.graphics', 'sci.med', 'talk.politics.guns']
newsgroups_train = fetch_20newsgroups(subset='all')

text_prep = Word2VecPrep(word2vec)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aleks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
train_x, train_y = text_prep.preprocess_dataset(newsgroups_train)
train_x.to(device)
train_y.to(device)
print(device)

c:\Users\aleks\Documents\Moje dokumenty\Studia\Praca Magisterska\NeuCube-Py-TextRep\experiments\preprocess.py:85: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  train_x = tensor(newsgroups_vectors)


cuda


In [6]:
#mask = (newsgroups_vectors_tensor > 0)
# mask = (newsgroups_vectors_tensor > -0.05) & (newsgroups_vectors_tensor < 0.05)
# data_x = torch.where(mask, torch.tensor(1), torch.tensor(0))
# data_x.to(device)

In [7]:
lsa_experiment(train_x, train_y, clf_type="regression", splits=8)

8it [02:26, 18.36s/it]



---- CLASSIFIER: regression ----
acc: 0.7152180834129258
[[400   4   1   0   2   2   1   3  33  12   0   8   0  13  17 195  16  62
   27   3]
 [  6 602  72  33  30 106  25   9   9  10   1  25  14   3  20   5   0   1
    2   0]
 [  4  69 621  89  34  96  24  12   7   4   2  10   6   0   4   0   1   0
    2   0]
 [  0  38  72 544 115  47  52  18   8   6   1  15  58   2   2   0   3   0
    1   0]
 [  0  35  46 142 535  42  51  10  15   2   1  10  60   3   8   0   1   1
    0   1]
 [  1  96  82  19  30 682  18   2   6   4   1  20   8   3  10   1   2   2
    1   0]
 [  1   6  17  47  14   6 745  35  22   9  10  11  31   3  10   1   3   0
    4   0]
 [  6   1   2   1   1   3  25 758  92   6   3   9  28   5   5   0  30   5
   10   0]
 [  1   6   1   1   4  12  17  95 792   4   6   2  13   9   6   2  12   3
   10   0]
 [  5   9   4   3   2   2  14   0  10 867  58   2   0   3   0   3   0   5
    7   0]
 [  4   2   1   2   0   3   2   0  14  18 941   5   0   1   0   1   1   1
    3   0]
 [ 10  

In [8]:
lsa_experiment(train_x, train_y, clf_type="random_forest", splits=8)

8it [10:37, 79.70s/it]



---- CLASSIFIER: random_forest ----
acc: 0.675952456754749
[[445   2   0   0   1   3   1   6  30  13   1  15   0  17  14 137  27  43
   24  20]
 [  9 561 100  33  27  91  46  13  10   4   2  23  14   4  26   3   3   2
    1   1]
 [  1  88 607  84  33  85  26   7  12   3   2  20   4   1   9   0   2   0
    0   1]
 [  0  43  68 530 121  40  69  12  11   3   3  12  56   4   6   0   2   0
    1   1]
 [  1  39  64 191 427  41  72  12  21   5   3  14  57   4   7   2   3   0
    0   0]
 [  4 119 114  35  32 569  41   2   9   3   3  23  10   6  11   2   3   0
    1   1]
 [  3  15  18  41  27  11 744  29  22   9  12   6  24   2   8   0   0   0
    4   0]
 [  6   4   3   9  15   7  42 702  99  14   4  13  28   2  11   0  22   3
    6   0]
 [  9   8   4   2   9   6  35 115 728  19   6   5  16   6   5   1  19   0
    3   0]
 [  5   8   3   1   2   1  19   3  13 820  96   6   0   2   3   0   3   2
    6   1]
 [  5   3   3   0   2   3   6   1  17  59 891   0   0   2   4   0   2   0
    1   0]
 [  9

In [9]:
lsa_experiment(train_x, train_y, clf_type="xgboost", splits=8)

8it [18:42, 140.31s/it]



---- CLASSIFIER: xgboost ----
acc: 0.7401040008489865
[[501   3   1   0   4   0   2   3  14   5   1  10   0  13  11  93  25  32
   24  57]
 [  6 615  72  30  30 100  25   4   9   7   2  14  21   4  21   3   1   4
    2   3]
 [  5  65 651  79  33  87  15   4   7   4   1   9  10   1   9   1   2   0
    2   0]
 [  0  47  77 561 141  28  42  11   5   2   4  10  42   6   1   1   2   0
    2   0]
 [  1  38  48 138 571  24  51  10  11   7   2   4  45   5   3   2   1   1
    1   0]
 [  0 106  73  31  32 668  18   1   7   0   3  10  12   3  10   5   3   3
    2   1]
 [  2  13  15  39  17   8 748  32  25   7   9   3  39   1   9   3   1   1
    2   1]
 [  2   7   7   8   7   5  25 747  87   5   7   8  36   5   5   4  14   1
    9   1]
 [  5   9   3   4   9   3  29  83 783   6   2   3  12   7   6   1  11   4
   15   1]
 [  6  10   3   5   0   4  14   2  13 848  60   3   5   2   3   0   1   3
    9   3]
 [  2   4   0   0   4   4   2   0  11  56 892   3   2   2   5   0   0   3
    7   2]
 [  8  23 

In [10]:
#encoder = Probability(iterations=100)
#spike_train_x = encoder.encode_dataset(newsgroups_vectors_tensor)

In [11]:
# snn_experiment(data_x, train_y, clf_type="regression", splits=4, shape=(10, 10, 10))